In [104]:
import numpy as np
def IntCode_Computer(IntCodeInitial, Opcodelocation, inputvalue, Relativebase = 0):
    IntCode = IntCodeInitial[:]
    def newlocation(location, facingdirection, output):
        #facingdirection: 0 = up, 1 = right, 2 = down, 3 = left
        if output == 0:
            facingdirection = (facingdirection - 1) % 4
        elif output == 1:
            facingdirection = (facingdirection + 1) % 4

        if facingdirection == 0:
            location[0] += 1
        elif facingdirection == 1:
            location[1] += 1
        elif facingdirection == 2:
            location[0] += -1
        elif facingdirection == 3:
            location[1] += -1

        return location, facingdirection

    def IncreaseMemory(intCode, maxlocation):
        if len(intCode) < maxlocation:
            intCode.extend([0] * (2000 - len(intCode)))
        return intCode

    def DetermineOpcodeNumber(number):
        return int(str(number)[-2:])
    
    def DetermineParameterArrayLocations(Intcode, Opcodelocation, Relativebase):
        def DetermineParameterMode(number):
            firstparameter = int(str(number // 100)[-1:])
            secondparameter = int(str(number // 1000)[-1:])
            thirdparameter = int(str(number // 10000)[-1:])
            return firstparameter, secondparameter, thirdparameter

        def DetermineParameterLocations(Intcode, Opcodelocation, parameters, Relativebase):
            # als parameter 0 is, dan verwijzing. Als parameter 1, dan eigen locatie
            firstparameterlocation = int(Opcodelocation + 1) if parameters[0] == 1 else int(Intcode[Opcodelocation + 1] + Relativebase) if parameters[0] == 2 else int(Intcode[Opcodelocation + 1])
            try:
                secondparameterlocation = int(Opcodelocation + 2) if parameters[1] == 1 else int(Intcode[Opcodelocation + 2] + Relativebase) if parameters[1] == 2 else int(Intcode[Opcodelocation + 2])
            except IndexError:
                secondparameterlocation = 0
            try:
                thirdparameterlocation = int(Opcodelocation + 3) if parameters[2] == 1 else int(Intcode[Opcodelocation + 3] + Relativebase) if parameters[2] == 2 else int(Intcode[Opcodelocation + 3])
            except IndexError:
                thirdparameterlocation = 0
            return firstparameterlocation, secondparameterlocation, thirdparameterlocation

        parameters = DetermineParameterMode(Intcode[Opcodelocation])
        return DetermineParameterLocations(Intcode, Opcodelocation, parameters, Relativebase)

    def RunOpcode(Intcode, OpcodeNumber, parameterlocations, inputvalue, Relativebase):
        def Opcode1(Intcode, parameterlocations, inputvalue, Relativebase):
            Intcode[parameterlocations[2]] = Intcode[parameterlocations[0]] + Intcode[parameterlocations[1]]
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode2(Intcode, parameterlocations, inputvalue, Relativebase):
            Intcode[parameterlocations[2]] = Intcode[parameterlocations[0]] * Intcode[parameterlocations[1]]
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode3(Intcode, parameterlocations, inputvalue, Relativebase):
            Intcode[parameterlocations[0]] = inputvalue
            return Intcode, Opcodelocation + 2, inputvalue, Relativebase
        def Opcode4(Intcode, parameterlocations, inputvalue, Relativebase):
            global counter
            global output
            counter += 1
            output = Intcode[parameterlocations[0]]
            #print(Intcode[parameterlocations[0]])
            return Intcode, Opcodelocation + 2, inputvalue, Relativebase
        def Opcode5(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] != 0:
                return Intcode, Intcode[parameterlocations[1]], inputvalue, Relativebase
            else:
                return Intcode, Opcodelocation + 3, inputvalue, Relativebase
        def Opcode6(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] == 0:
                return Intcode, Intcode[parameterlocations[1]], inputvalue, Relativebase
            else:
                return Intcode, Opcodelocation + 3, inputvalue, Relativebase
        def Opcode7(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] < Intcode[parameterlocations[1]]:
                Intcode[parameterlocations[2]] = 1
            else:
                Intcode[parameterlocations[2]] = 0
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode8(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] == Intcode[parameterlocations[1]]:
                Intcode[parameterlocations[2]] = 1
            else:
                Intcode[parameterlocations[2]] = 0
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode9(Intcode, parameterlocations, inputvalue, Relativebase):
            Relativebase += Intcode[parameterlocations[0]]
            return Intcode, Opcodelocation + 2, inputvalue, Relativebase
        
        Intcode = IncreaseMemory(IntCode[:], (max(parameterlocations)))
        if OpcodeNumber == 1:
            return Opcode1(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 2:
            return Opcode2(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 3:
            return Opcode3(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 4:
            return Opcode4(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 5:
            return Opcode5(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 6:
            return Opcode6(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 7:
            return Opcode7(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 8:
            return Opcode8(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 9:
            return Opcode9(Intcode, parameterlocations, inputvalue, Relativebase)

    global counter
    location = [0,0]
    locationcolor = {(location[0], location[1]): 0}
    facingdirection = 0
    while IntCode[Opcodelocation] != 99:
        IntCode, Opcodelocation, inputvalue, Relativebase = RunOpcode(IntCode, DetermineOpcodeNumber(IntCode[Opcodelocation]), DetermineParameterArrayLocations(IntCode, Opcodelocation, Relativebase), inputvalue, Relativebase)[:]
        if counter == 1:
            locationcolor[(location[0], location[1])] = output
        elif counter == 2:
            location, facingdirection = newlocation(location, facingdirection, output)
            if (location[0],location[1]) in locationcolor:
                inputvalue = locationcolor[(location[0],location[1])]
            else:
                inputvalue = 0 # 0 voor zwart begin, 1 voor wit begin
            counter = 0
            
            
    return [IntCode, -999, inputvalue, locationcolor]

In [105]:
inputcode = [3,8,1005,8,305,1106,0,11,0,0,0,104,1,104,0,3,8,1002,8,-1,10,101,1,10,10,4,10,1008,8,0,10,4,10,1002,8,1,29,3,8,102,-1,8,10,1001,10,1,10,4,10,108,1,8,10,4,10,1002,8,1,50,1,104,20,10,1,1102,6,10,1006,0,13,3,8,102,-1,8,10,101,1,10,10,4,10,108,1,8,10,4,10,102,1,8,83,1,1102,0,10,1006,0,96,2,1004,19,10,3,8,1002,8,-1,10,101,1,10,10,4,10,108,0,8,10,4,10,101,0,8,116,3,8,1002,8,-1,10,1001,10,1,10,4,10,108,1,8,10,4,10,102,1,8,138,1006,0,60,1,1008,12,10,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,102,1,8,168,1006,0,14,1006,0,28,3,8,1002,8,-1,10,1001,10,1,10,4,10,108,0,8,10,4,10,101,0,8,195,2,1005,9,10,1006,0,29,3,8,1002,8,-1,10,101,1,10,10,4,10,108,1,8,10,4,10,1002,8,1,224,2,1009,8,10,1,3,5,10,3,8,1002,8,-1,10,101,1,10,10,4,10,108,1,8,10,4,10,102,1,8,254,3,8,102,-1,8,10,1001,10,1,10,4,10,1008,8,0,10,4,10,1002,8,1,277,1,1003,18,10,1,1104,1,10,101,1,9,9,1007,9,957,10,1005,10,15,99,109,627,104,0,104,1,21101,0,666681062292,1,21102,322,1,0,1105,1,426,21101,847073883028,0,1,21102,333,1,0,1105,1,426,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,21101,0,179356855319,1,21102,1,380,0,1105,1,426,21102,1,179356998696,1,21102,1,391,0,1105,1,426,3,10,104,0,104,0,3,10,104,0,104,0,21101,0,988669698816,1,21101,0,414,0,1106,0,426,21102,1,868494500628,1,21102,425,1,0,1106,0,426,99,109,2,21202,-1,1,1,21102,1,40,2,21102,457,1,3,21102,1,447,0,1105,1,490,109,-2,2105,1,0,0,1,0,0,1,109,2,3,10,204,-1,1001,452,453,468,4,0,1001,452,1,452,108,4,452,10,1006,10,484,1102,0,1,452,109,-2,2105,1,0,0,109,4,1201,-1,0,489,1207,-3,0,10,1006,10,507,21102,0,1,-3,22101,0,-3,1,21202,-2,1,2,21101,1,0,3,21102,1,526,0,1106,0,531,109,-4,2105,1,0,109,5,1207,-3,1,10,1006,10,554,2207,-4,-2,10,1006,10,554,22101,0,-4,-4,1106,0,622,21201,-4,0,1,21201,-3,-1,2,21202,-2,2,3,21102,573,1,0,1106,0,531,21202,1,1,-4,21101,1,0,-1,2207,-4,-2,10,1006,10,592,21102,1,0,-1,22202,-2,-1,-2,2107,0,-3,10,1006,10,614,22101,0,-1,1,21102,614,1,0,105,1,489,21202,-2,-1,-2,22201,-4,-2,-4,109,-5,2105,1,0]

In [106]:
#vraag 1
global counter
counter = 0
global output
output = IntCode_Computer(inputcode,0,0)

print(len(output[3]))


2041


In [107]:
import numpy as np
def IntCode_Computer(IntCodeInitial, Opcodelocation, inputvalue, Relativebase = 0):
    IntCode = IntCodeInitial[:]
    def newlocation(location, facingdirection, output):
        #facingdirection: 0 = up, 1 = right, 2 = down, 3 = left
        if output == 0:
            facingdirection = (facingdirection - 1) % 4
        elif output == 1:
            facingdirection = (facingdirection + 1) % 4

        if facingdirection == 0:
            location[0] += 1
        elif facingdirection == 1:
            location[1] += 1
        elif facingdirection == 2:
            location[0] += -1
        elif facingdirection == 3:
            location[1] += -1

        return location, facingdirection

    def IncreaseMemory(intCode, maxlocation):
        if len(intCode) < maxlocation:
            intCode.extend([0] * (2000 - len(intCode)))
        return intCode

    def DetermineOpcodeNumber(number):
        return int(str(number)[-2:])
    
    def DetermineParameterArrayLocations(Intcode, Opcodelocation, Relativebase):
        def DetermineParameterMode(number):
            firstparameter = int(str(number // 100)[-1:])
            secondparameter = int(str(number // 1000)[-1:])
            thirdparameter = int(str(number // 10000)[-1:])
            return firstparameter, secondparameter, thirdparameter

        def DetermineParameterLocations(Intcode, Opcodelocation, parameters, Relativebase):
            # als parameter 0 is, dan verwijzing. Als parameter 1, dan eigen locatie
            firstparameterlocation = int(Opcodelocation + 1) if parameters[0] == 1 else int(Intcode[Opcodelocation + 1] + Relativebase) if parameters[0] == 2 else int(Intcode[Opcodelocation + 1])
            try:
                secondparameterlocation = int(Opcodelocation + 2) if parameters[1] == 1 else int(Intcode[Opcodelocation + 2] + Relativebase) if parameters[1] == 2 else int(Intcode[Opcodelocation + 2])
            except IndexError:
                secondparameterlocation = 0
            try:
                thirdparameterlocation = int(Opcodelocation + 3) if parameters[2] == 1 else int(Intcode[Opcodelocation + 3] + Relativebase) if parameters[2] == 2 else int(Intcode[Opcodelocation + 3])
            except IndexError:
                thirdparameterlocation = 0
            return firstparameterlocation, secondparameterlocation, thirdparameterlocation

        parameters = DetermineParameterMode(Intcode[Opcodelocation])
        return DetermineParameterLocations(Intcode, Opcodelocation, parameters, Relativebase)

    def RunOpcode(Intcode, OpcodeNumber, parameterlocations, inputvalue, Relativebase):
        def Opcode1(Intcode, parameterlocations, inputvalue, Relativebase):
            Intcode[parameterlocations[2]] = Intcode[parameterlocations[0]] + Intcode[parameterlocations[1]]
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode2(Intcode, parameterlocations, inputvalue, Relativebase):
            Intcode[parameterlocations[2]] = Intcode[parameterlocations[0]] * Intcode[parameterlocations[1]]
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode3(Intcode, parameterlocations, inputvalue, Relativebase):
            Intcode[parameterlocations[0]] = inputvalue
            return Intcode, Opcodelocation + 2, inputvalue, Relativebase
        def Opcode4(Intcode, parameterlocations, inputvalue, Relativebase):
            global counter
            global output
            counter += 1
            output = Intcode[parameterlocations[0]]
            #print(Intcode[parameterlocations[0]])
            return Intcode, Opcodelocation + 2, inputvalue, Relativebase
        def Opcode5(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] != 0:
                return Intcode, Intcode[parameterlocations[1]], inputvalue, Relativebase
            else:
                return Intcode, Opcodelocation + 3, inputvalue, Relativebase
        def Opcode6(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] == 0:
                return Intcode, Intcode[parameterlocations[1]], inputvalue, Relativebase
            else:
                return Intcode, Opcodelocation + 3, inputvalue, Relativebase
        def Opcode7(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] < Intcode[parameterlocations[1]]:
                Intcode[parameterlocations[2]] = 1
            else:
                Intcode[parameterlocations[2]] = 0
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode8(Intcode, parameterlocations, inputvalue, Relativebase):
            if Intcode[parameterlocations[0]] == Intcode[parameterlocations[1]]:
                Intcode[parameterlocations[2]] = 1
            else:
                Intcode[parameterlocations[2]] = 0
            return Intcode, Opcodelocation + 4, inputvalue, Relativebase
        def Opcode9(Intcode, parameterlocations, inputvalue, Relativebase):
            Relativebase += Intcode[parameterlocations[0]]
            return Intcode, Opcodelocation + 2, inputvalue, Relativebase
        
        Intcode = IncreaseMemory(IntCode[:], (max(parameterlocations)))
        if OpcodeNumber == 1:
            return Opcode1(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 2:
            return Opcode2(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 3:
            return Opcode3(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 4:
            return Opcode4(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 5:
            return Opcode5(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 6:
            return Opcode6(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 7:
            return Opcode7(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 8:
            return Opcode8(Intcode, parameterlocations, inputvalue, Relativebase)
        elif OpcodeNumber == 9:
            return Opcode9(Intcode, parameterlocations, inputvalue, Relativebase)

    global counter
    location = [0,0]
    locationcolor = {(location[0], location[1]): 0}
    facingdirection = 0
    while IntCode[Opcodelocation] != 99:
        IntCode, Opcodelocation, inputvalue, Relativebase = RunOpcode(IntCode, DetermineOpcodeNumber(IntCode[Opcodelocation]), DetermineParameterArrayLocations(IntCode, Opcodelocation, Relativebase), inputvalue, Relativebase)[:]
        if counter == 1:
            locationcolor[(location[0], location[1])] = output
        elif counter == 2:
            location, facingdirection = newlocation(location, facingdirection, output)
            if (location[0],location[1]) in locationcolor:
                inputvalue = locationcolor[(location[0],location[1])]
            else:
                inputvalue = 1 # 0 voor zwart begin, 1 voor wit begin
            counter = 0
            
            
    return [IntCode, -999, inputvalue, locationcolor]

In [109]:
#vraag 2
global counter
counter = 0
global output
output = IntCode_Computer(inputcode,0,1)

print(output[3])

{(0, 0): 0, (0, 1): 1, (-1, 1): 0, (-1, 2): 0, (0, 2): 1, (0, 3): 1, (-1, 3): 0, (-1, 4): 1, (0, 4): 1, (0, 5): 0, (-1, 5): 0, (-1, 6): 1, (0, 6): 1, (0, 7): 1, (-1, 7): 0, (-1, 8): 0, (0, 8): 1, (0, 9): 0, (-1, 9): 1, (-1, 10): 0, (0, 10): 0, (0, 11): 1, (-1, 11): 0, (-1, 12): 0, (0, 12): 1, (0, 13): 1, (-1, 13): 0, (-1, 14): 1, (0, 14): 1, (0, 15): 0, (-1, 15): 0, (-1, 16): 1, (0, 16): 1, (0, 17): 1, (-1, 17): 0, (-1, 18): 0, (0, 18): 1, (0, 19): 0, (-1, 19): 1, (-1, 20): 0, (0, 20): 0, (0, 21): 1, (-1, 21): 1, (-1, 22): 0, (0, 22): 0, (0, 23): 0, (-1, 23): 1, (-1, 24): 0, (0, 24): 1, (0, 25): 0, (-1, 25): 0, (-1, 26): 1, (0, 26): 1, (0, 27): 1, (-1, 27): 0, (-1, 28): 0, (0, 28): 1, (0, 29): 1, (-1, 29): 0, (-1, 30): 0, (0, 30): 0, (0, 31): 1, (-1, 31): 0, (-1, 32): 0, (0, 32): 1, (0, 33): 1, (-1, 33): 0, (-1, 34): 1, (0, 34): 1, (0, 35): 0, (-1, 35): 0, (-1, 36): 1, (0, 36): 1, (0, 37): 1, (-1, 37): 0, (-1, 38): 0, (0, 38): 1, (0, 39): 0, (-1, 39): 1, (-1, 40): 0, (0, 40): 0, (0, 41

In [110]:
x, y, z = np.array([(x, y, z) for (x, y), z in output[3].items()]).T

In [113]:
print(x,y,z)

[ 0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1
  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1
  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1
  0  0 -1 -1  0  0 -1 -1  0  0 -1 -1 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3
 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3
 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3
 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3 -2 -2 -3 -3
 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5
 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5
 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5 -4 -4 -5 -5
 -4 -4 -5 -5 -4 -4 -5 -5 -4] [ 0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12
 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23 24
 24 25 25 26 26 27 27 28 28 29 29 30 30 31 31 32 32 33 33 34 34 35 35 36
 36 37 37 38 38 39 39 

In [132]:
from PIL import Image

img = Image.new('1', (10,50))
width = 10*300
height = 50*300
pixels = img.load()

for j in range(0,len(z)):
    xcoord = int(x[j]+5)
    ycoord = int(y[j]+1)
    color = int(z[j])

    pixels[xcoord,ycoord] = color

img = img.resize((width, height), Image.ANTIALIAS)

img.show()